# Multivariate Poisson CS

In [1]:
using Revise

In [2]:
using GLMCopula, DelimitedFiles, LinearAlgebra, Random, GLM, MixedModels, CategoricalArrays
using Random, Roots, SpecialFunctions, LoopVectorization
using DataFrames, DelimitedFiles, Statistics, ToeplitzMatrices
using BenchmarkTools, Test
import StatsBase: sem

p = 3    # number of fixed effects, including intercept

# true parameter values
Random.seed!(12345)
βtrue = rand(Uniform(-2, 2), p)
σ2true = [0.5]
ρtrue = [0.5]

function get_V(ρ, n)
    vec = zeros(n)
    vec[1] = 1.0
    for i in 2:n
        vec[i] = ρ
    end
    V = ToeplitzMatrices.SymmetricToeplitz(vec)
    V
end

#simulation parameters
samplesize = 10000

st = time()
currentind = 1
d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64

gcs = Vector{GLMCopulaCSObs{T, D, Link}}(undef, samplesize)

ni = 5 #  number of observations per individual
V = get_V(ρtrue[1], ni)

# true Gamma
Γ = σ2true[1] * V

# for reproducibility I will simulate all the design matrices here
Random.seed!(12345)
X_samplesize = [randn(ni, p - 1) for i in 1:samplesize]

for i in 1:samplesize
    X = [ones(ni) X_samplesize[i]]
    η = X * βtrue
    μ = exp.(η)
    vecd = Vector{DiscreteUnivariateDistribution}(undef, ni)
    for i in 1:ni
        vecd[i] = Poisson(μ[i])
    end
    nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
    # simuate single vector y
    y = Vector{Float64}(undef, ni)
    res = Vector{Float64}(undef, ni)
    rand(nonmixed_multivariate_dist, y, res)
    # push!(Ystack, y)
    V = [Float64.(Matrix(I, ni, ni))]
    # V = [ones(ni, ni)]
    gcs[i] = GLMCopulaCSObs(y, X, d, link)
end

# form model
gcm = GLMCopulaCSModel(gcs);

initialize_model!(gcm)
@show gcm.β
@show gcm.ρ
@show gcm.σ2;

┌ Info: Precompiling GLMCopula [c47b6ae2-b804-4668-9957-eb588c99ffbc]
└ @ Base loading.jl:1342


initializing β using Newton's Algorithm under Independence Assumption
initializing σ2 and ρ using method of moments
ρhat = 0.0030790202693197176
σ2hat = 46.91866084747433
gcm.β = [0.3377202759458189, 1.3605300243381582, -0.4998165016998886]
gcm.ρ = [0.0030790202693197176]
gcm.σ2 = [0.18343216493113093]


In [3]:
gc = gcm.data[1]
β = gcm.β
ρ = gcm.ρ[1]
σ2 = gcm.σ2[1]

0.18343216493113093

### check each function within logl

In [4]:
using BenchmarkTools
@benchmark get_V!($ρ, $gc)

BenchmarkTools.Trial: 10000 samples with 973 evaluations.
 Range (min … max):  59.318 ns … 260.259 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     63.481 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   68.709 ns ±  17.931 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▇█▅    ▁▂▂                                                  ▁
  ████▇▇▆▆███▇█▇▆▇▇▇▇█▇█▇▇▇▇▆▇▆▇▆▇▆▅▄▅▄▅▅▅▅▅▆▆▆▄▅▄▅▅▅▅▆▅▄▅▃▄▅▄ █
  59.3 ns       Histogram: log(frequency) by time       157 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [5]:
@benchmark get_∇V!($gc)

BenchmarkTools.Trial: 10000 samples with 972 evaluations.
 Range (min … max):  59.099 ns … 242.647 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     63.479 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   67.955 ns ±  15.837 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▆██▄     ▂▄▁         ▁ ▁                                    ▁
  ██████▇▆▄▇███▇▇▇▇▅█▆█▆█▆█▅▆▇▇█▇▆▆▆▆▇▆▆▆▄▅▄▄▆▅▆▅▆▅▆▅▄▆▄▅▅▅▅▄▄ █
  59.1 ns       Histogram: log(frequency) by time       145 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [6]:
@benchmark update_res!($gc, $β)

BenchmarkTools.Trial: 10000 samples with 921 evaluations.
 Range (min … max):  111.378 ns … 349.670 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     114.781 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   122.192 ns ±  19.984 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃█▅▄▅▃▄▄▁      ▃                                              ▁
  ██████████▇▇▇███▇▆▃▆▅▆▇▆▇▆▇▇▅▆█▆▆█▄▅▆▆▆▆▆▄▅▄▅▄▄▄▄▆▅▄▄▃▄▄▃▄▃▄▄ █
  111 ns        Histogram: log(frequency) by time        224 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [7]:
@benchmark standardize_res!($gc)

BenchmarkTools.Trial: 10000 samples with 997 evaluations.
 Range (min … max):  126.815 ns … 411.689 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     130.806 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   141.408 ns ±  30.569 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█▂▂▃    ▁ ▄      ▁                                           ▁
  █████▇▇▇▇█▇██▇█████▇▇▇▆█▇▇█▇▇▇▇▇▆▆▇▇▆▆▆▆▆▆▆▆▆▆▅▅▅▆▅▅▅▅▅▅▅▄▄▅▅ █
  127 ns        Histogram: log(frequency) by time        282 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [8]:
@benchmark std_res_differential!($gc)

BenchmarkTools.Trial: 10000 samples with 204 evaluations.
 Range (min … max):  379.775 ns …  1.258 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     388.623 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   407.225 ns ± 69.730 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▆▄▃      ▂ ▄▂                                              ▁
  ██████▅▇▆▇▆████▇▅▅▄▅▄▅▅▅▆▆▆▄▅▆▆▇▇▅▅▅▄▅▃▄▅▄▅▄▅▅▄▄▄▅▅▃▄▃▄▅▃▄▄▅ █
  380 ns        Histogram: log(frequency) by time       765 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [9]:
@benchmark GLMCopula.component_loglikelihood($gc)

BenchmarkTools.Trial: 10000 samples with 932 evaluations.
 Range (min … max):  109.065 ns … 393.635 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     117.725 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   118.809 ns ±  16.670 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▂▂▄▂▆▆▄▆▁▂    ▂ ▂                                           ▂
  ████████████▇▇▆▇█▇██▆▆▅▃▅▅▃▆▄▅▅▆▅▅▇▆▄▄█▅▅▅█▄▄▄▅▃▄▄▄▄▄▄▄▂▅▄▄▃▄ █
  109 ns        Histogram: log(frequency) by time        194 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [10]:
@benchmark GLMCopula.glm_gradient($gc)

BenchmarkTools.Trial: 10000 samples with 195 evaluations.
 Range (min … max):  484.026 ns …  1.610 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     487.997 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   506.610 ns ± 77.410 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▃           ▁ ▂                                             ▁
  ██▇▅▅▅▃▆▃▆▅▇▅█▇██▆▆▅▆▅▅▆▅▆▆▆▅▇▅▇█▇▅▄█▃▄▆▆▅▅▄▄▅▅▄▅▄▃▄▅▅▁▅▃▄▄▄ █
  484 ns        Histogram: log(frequency) by time       909 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [11]:
@benchmark GLMCopula.glm_hessian($gc)

BenchmarkTools.Trial: 10000 samples with 818 evaluations.
 Range (min … max):  147.767 ns … 420.790 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     155.061 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   157.234 ns ±  18.134 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ ▅ ▇▆ ▆  ▅        ▂  ▂                                       ▂
  ███▇██▅██▄█▆▇█▇▇▇▇██▆▆█▄▄▅▄▄▄▅▄▅▄▆▅▄▅▅▅▄▄▃▇▄▅▃█▆▅▅▃▇▇▃▃▅▄▆▅▄▄ █
  148 ns        Histogram: log(frequency) by time        238 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# with just the loglikelihood

In [12]:
@benchmark loglikelihood!($gc, $β, $ρ, $σ2, false, false)

BenchmarkTools.Trial: 10000 samples with 198 evaluations.
 Range (min … max):  437.354 ns …  1.511 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     439.338 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   470.291 ns ± 91.269 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄▅▂▃▃    ▂▂                                                 ▁
  ██████▆▅▆▇██▇▇▄▆▄▆▅▆▆▅▇▅█▆▆█▆▅▇▆▄▄▅▅▅▅▄▅▆▅▅▅▄▄▄▅▅▅▅▃▄▅▄▅▃▅▄▄ █
  437 ns        Histogram: log(frequency) by time       937 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# with logl and gradient

In [13]:
@benchmark loglikelihood!($gc, $β, $ρ, $σ2, true, false)

BenchmarkTools.Trial: 10000 samples with 145 evaluations.
 Range (min … max):  698.538 ns …   2.121 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     704.124 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   739.335 ns ± 108.536 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▁▃ ▄ ▄     ▁  ▂                                             ▁
  ████▇█▇█▇▇▅▇▆█▆▆█▅▆▄▄▆▄▅▅▅▄▅▅▅▇▆▅▇▆▄▄▇▃▄▄▄▄▄▅▃▅▄▃▂▃▄▃▄▃▄▄▄▃▅▅ █
  699 ns        Histogram: log(frequency) by time       1.29 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.